# Experimenting with the AMASS dataset

In [1]:
import os.path as osp

import torch
import numpy as np

from human_body_prior.tools.omni_tools import copy2cpu as c2c

support_dir = '/home/kaseris/Documents/amass/support_data'
comp_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [27]:
amass_npz_fname = osp.join(support_dir, 'github_data/dmpl_sample.npz')
bdata = np.load(amass_npz_fname)

subject_gender = bdata['gender'].tostring().decode('utf-8')

/tmp/ipykernel_680365/1067486171.py:4: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  subject_gender = bdata['gender'].tostring().decode('utf-8')


Let's see what's inside the bdata

- What are the `dmpls`? Some body parameters????
- What are the `betas`? Body parameters

In [28]:
print(f'The available keys from the read file are: {list(bdata.keys())}')
print(f'betas: {bdata["betas"].shape}')
print(f'dmpls: {bdata["dmpls"].shape}')
print(f'The subject gender is: {subject_gender}')

The available keys from the read file are: ['poses', 'gender', 'mocap_framerate', 'betas', 'marker_data', 'dmpls', 'marker_labels', 'trans']
betas: (16,)
dmpls: (235, 8)
The subject gender is: female


In [29]:
from human_body_prior.body_model.body_model import BodyModel

dmpl_fname = osp.join(support_dir, f'body_models/dmpls/{subject_gender}/model.npz')
bm_fname = osp.join(support_dir, f'body_models/smplh/{subject_gender}/model.npz')

num_betas = 16 # number of body parameters
num_dmpls = 8 # number of DMPL parameters

bm = BodyModel(bm_fname=bm_fname, num_betas=num_betas, num_dmpls=num_dmpls, dmpl_fname=dmpl_fname).to(comp_device)
faces = c2c(bm.f)

The provided sample data has the original mocap marker data.

In the real AMASS dataset, we include only markers for the test set.

For the rest of the subsets you can obtain the marker data from their respective websites.

In the following we make PyTorch tensors for parameters controlling different part of the body model.

In [33]:
time_length = len(bdata['trans'])
body_parms = {
    'root_orient': torch.Tensor(bdata['poses'][:, :3]).to(comp_device), # controls the global root orientation
    'pose_body': torch.Tensor(bdata['poses'][:, 3:66]).to(comp_device), # controls the body
    'pose_hand': torch.Tensor(bdata['poses'][:, 66:]).to(comp_device), # controls the finger articulation
    'trans': torch.Tensor(bdata['trans']).to(comp_device), # controls the global body position
    'betas': torch.Tensor(np.repeat(bdata['betas'][:num_betas][np.newaxis], repeats=time_length, axis=0)).to(comp_device), # controls the body shape. Body shape is static
    'dmpls': torch.Tensor(bdata['dmpls'][:, :num_dmpls]).to(comp_device) # controls soft tissue dynamics
}
print('Body parameter vector shapes: \n{}'.format(' \n'.join(['{}: {}'.format(k,v.shape) for k,v in body_parms.items()])))
print('time_length = {}'.format(time_length))

Body parameter vector shapes: 
root_orient: torch.Size([235, 3]) 
pose_body: torch.Size([235, 63]) 
pose_hand: torch.Size([235, 90]) 
trans: torch.Size([235, 3]) 
betas: torch.Size([235, 16]) 
dmpls: torch.Size([235, 8])
time_length = 235
